In [70]:
import pandas as pd

### IPS 
as "Indice de Position Social" is a index calculated from socio-professional categories of both parents and legal representative, revenues of parents, material situation, educational intrest and cultural implication.
It scales from 38 to 179 (no unit) and the actual formula to calculate it has not been made public. We possess the average IPS of nearly every highschools in France. 
IPS is a mean to observe and fight against school related inequalities.
### Ival
as "Indicateur de Valeur ajoutée" is a number (positive or negative, no unit) calculated from the actual result minus the expected result. This value shows if a school is capable of supporting their students from entry to graduation. 

In [71]:
#Dataframe containing ips of high schools in france
ips_df = pd.read_csv("../data/ips.csv", delimiter=";")

#Dataframe containing exam results of high schools in france
res_df = pd.read_csv("../data/results.csv", delimiter=";", low_memory=False)

In [72]:
#Taking out values = 'ND' since it is a string and does not appear frequently
res_df = res_df[res_df['Valeur ajoutee du taux de réussite - Gnle'] != 'ND']
res_df = res_df[res_df['Valeur ajoutee du taux de reussite - Toutes series'] != 'ND']
#Taking out values taken before 2017 since ips_df does not have values (for exams which take place at the last year of the school year) before 2018
res_df = res_df[res_df["Annee"] >= 2017]

#Converting the year range "20XX(0) - 20XX(1)" to "20XX(1)"
ips_df["Rentrée scolaire"] = ips_df["Rentrée scolaire"].apply(lambda x: x.split(sep="-")[1])

In [73]:
#Convertting values for merge (avoid ValueError)
for col in ips_df.columns:
    ips_df[col] = ips_df[col].apply(str)

for col in res_df.columns:
    res_df[col] = res_df[col].apply(str)

In [74]:
#Merging columns of all exam types, did not have time to figure our a better way to do this

#Seperating res_df by year
res_df_year = res_df.groupby(['Annee'])

#Seperating res_df by type of exam
res_df_tt = pd.DataFrame()
res_df_gn = pd.DataFrame()
for year in res_df_year.groups:
    if int(year) < 2021:
        res_df_tt = pd.concat([res_df_tt, res_df_year.get_group(year)], sort=False)
    else:
        res_df_gn = pd.concat([res_df_gn, res_df_year.get_group(year)], sort=False)
        
#Merging res_df_tt and res_df_gn
res_df_tt.drop(columns=["Presents - Gnle", "Taux de reussite - Gnle", "Taux de mentions - Gnle", "Valeur ajoutee du taux de réussite - Gnle", "Valeur ajoutée du taux de mentions - Gnle"], inplace=True)
res_df_tt.rename(columns={"Presents - Toutes series" : "Presents", "Taux de reussite - Toutes series" : "Taux de reussite", "Taux de mentions - Toutes series" : "Taux de mentions", "Valeur ajoutee du taux de reussite - Toutes series" : "Valeur ajoutee du taux de reussite", "Valeur ajoutee du taux de mentions - Toutes series" : "Valeur ajoutee du taux de mentions"}, inplace=True)
res_df_gn.drop(columns=["Presents - Toutes series", "Taux de reussite - Toutes series", "Taux de mentions - Toutes series", "Valeur ajoutee du taux de reussite - Toutes series", "Valeur ajoutee du taux de mentions - Toutes series"], inplace=True)
res_df_gn.rename(columns={"Presents - Gnle" : "Presents", "Taux de reussite - Gnle" : "Taux de reussite", "Taux de mentions - Gnle" : "Taux de mentions", "Valeur ajoutee du taux de réussite - Gnle" : "Valeur ajoutee du taux de reussite", "Valeur ajoutee du taux de mentions - Gnle" : "Valeur ajoutee du taux de mentions"}, inplace=True)

res_df = pd.concat([res_df_tt, res_df_gn], sort=False)

In [75]:
ips_df = ips_df.rename(columns={"Rentrée scolaire":"Annee"})
#Left to ensure all data is present
ips_res_df = pd.merge(ips_df, res_df, how="left", on=["UAI", "Annee", "Secteur"])

In [76]:
#Creating a new df with only useful columns
ips_res_df = ips_res_df[["Annee", "Code region", "Code departement", "Code commune", "UAI", "Secteur", "Type de lycée", 
                        "IPS voie GT", "IPS voie PRO", "IPS Ensemble GT-PRO",
                        "Ecart-type de l'IPS voie GT", "Ecart-type de l'IPS voie PRO", 
                        "Presents", "Taux de reussite", "Taux de mentions",
                        "Valeur ajoutee du taux de mentions", "Valeur ajoutee du taux de reussite",
                        "Effectif de seconde", "Effectif de premiere", "Effectif de terminale",
                        "Taux d'acces 2nde-bac", "Taux d'acces 1ere-bac", "Taux d'acces terminale-bac",
                        "Valeur ajoutee du taux d'acces 2nde-bac", "Valeur ajoutee du taux d'acces 1ere-bac", "Valeur ajoutee du taux d'acces terminale-bac"
                        ]]

ips_res_df.to_csv("../data/ips_results.csv")